In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.preprocessing as skp
from sklearn.model_selection import train_test_split
import tensorflow.keras as tfk
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score

In [78]:
from sklearn.pipeline import Pipeline, make_pipeline, FunctionTransformer
from sklearn.impute import SimpleImputer

In [79]:
dataset = pd.read_csv("./diabetes.csv")

In [80]:
target = dataset["Outcome"]
# features = dataset.drop(columns=["Outcome"],axis=1)
features = dataset.drop(columns=["Outcome","BloodPressure","SkinThickness"],axis=1)

In [81]:
from sklearn.base import TransformerMixin, BaseEstimator

class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):

        # Here you just print what you need + return the actual data. You're not transforming anything. 

        print("Shape of Pre-processed Data:", data.shape)
        return data

    def fit(self, data, y=None, **fit_params):

        # No need to fit anything, because this is not an actual  transformation. 

        return self

In [82]:
scaler = skp.StandardScaler()

In [83]:
# from sklearn import svm
# model = svm.SVC(kernel="sigmoid")

In [84]:
model = tfk.Sequential([
    layers.InputLayer(shape=(6,)),
    layers.Dense(4,activation="relu"),
    layers.Dense(2,activation="relu"),
    layers.Dense(1,activation="sigmoid")
])
model.compile(
    optimizer="adam",
    loss = tfk.losses.BinaryCrossentropy(),
    metrics=["Accuracy"]
)

In [85]:
def drop_column_series_dataframe(input):
    col_list = ["BloodPressure","SkinThickness"]
    input.drop(columns = col_list, axis=1,inplace=True)

In [86]:
drop_redundant_features_transformer = FunctionTransformer(drop_column_series_dataframe)

In [87]:
# preprocessor = make_pipeline(drop_redundant_features_transformer, scaler)
preprocessor = make_pipeline(scaler)

In [88]:
preprocessor.fit(features)

Pipeline(steps=[('standardscaler', StandardScaler())])

In [89]:
model_pipeline = Pipeline(steps=[
    ("preprocessor",preprocessor),
    ("Debugger", Debugger()),
    ("model",model)
])

In [90]:
target = np.asarray(target).reshape(-1,1)

In [91]:
features

,Pregnancies,Glucose,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,0,33.6,0.627,50
1,1,85,0,26.6,0.351,31
2,8,183,0,23.3,0.672,32
3,1,89,94,28.1,0.167,21
4,0,137,168,43.1,2.288,33
...,...,...,...,...,...,...
763,10,101,180,32.9,0.171,63
764,2,122,0,36.8,0.340,27
765,5,121,112,26.2,0.245,30
766,1,126,0,30.1,0.349,47


In [92]:
model_pipeline.fit(features,target)

Shape of Pre-processed Data: (768, 6)


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - Accuracy: 0.3133 - loss: 0.8864


Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('standardscaler', StandardScaler())])),
                ('Debugger', Debugger()),
                ('model', <Sequential name=sequential_5, built=True>)])

In [96]:
model_pipeline.predict([[6,148,0,33.6,0.627,50],[1,85,0,26.6,0.351,31],[0,118,230,45.8,0.551,31]])

Shape of Pre-processed Data: (3, 6)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


/root/Diabetes Prediction ML/Diabetes/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[0.49426815],
       [0.554909  ],
       [0.8742109 ]], dtype=float32)